# Imports

In [39]:
import pandas as pd
import datetime as dt
# from datetime import datetime, timezone
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
import xgboost as xg
from entsoe import EntsoePandasClient
from statsmodels.graphics import tsaplots

In [40]:
from helper import *

# Import data

## Entsoe

In [41]:
day_ahead_prices = pd.read_csv('./data/day_ahead_prices.csv')
day_ahead_prices.tail()

,Unnamed: 0,Day-ahead prices
43796,2022-12-30 20:00:00+01:00,19.88
43797,2022-12-30 21:00:00+01:00,1.45
43798,2022-12-30 22:00:00+01:00,0.61
43799,2022-12-30 23:00:00+01:00,0.01
43800,2022-12-31 00:00:00+01:00,2.00


In [42]:
load_and_forecast = pd.read_csv('./data/load_and_forecast.csv')
load_and_forecast.tail()

,Unnamed: 0,Forecasted Load,Actual Load
43793,2022-12-30 19:00:00+01:00,4344.0,4375.0
43794,2022-12-30 20:00:00+01:00,4179.0,4187.0
43795,2022-12-30 21:00:00+01:00,4053.0,4103.0
43796,2022-12-30 22:00:00+01:00,3889.0,3965.0
43797,2022-12-30 23:00:00+01:00,3656.0,3750.0


In [43]:
wind_solar_forecast = pd.read_csv('./data/wind_solar_forecast.csv')
wind_solar_forecast.tail()

,Unnamed: 0,Solar,Wind Offshore,Wind Onshore
43699,2022-12-30 19:00:00+01:00,0.0,1445.0,2973.0
43700,2022-12-30 20:00:00+01:00,0.0,1436.0,3033.0
43701,2022-12-30 21:00:00+01:00,0.0,1406.0,3205.0
43702,2022-12-30 22:00:00+01:00,0.0,1417.0,3272.0
43703,2022-12-30 23:00:00+01:00,0.0,1422.0,3275.0


In [64]:
imports = pd.read_csv('./data/imports.csv')
imports = remove_utc('Unnamed: 0', 1, imports)
importshour = imports.groupby(pd.Grouper(freq='H')).sum()

In [65]:
exDELU = pd.read_csv('./data/exportsDELU.csv')
exDK1 = pd.read_csv('./data/exportsDK1.csv')
exSE4 = pd.read_csv('./data/exportsSE4.csv')
exDELUAT = pd.read_csv('./data/exportsDELUAT.csv')

In [66]:
# Apply Time Removal Function and makes it a Timestamp
exDELU = remove_utc('Unnamed: 0', 1, exDELU)
exDK1 = remove_utc('Unnamed: 0', 1, exDK1)
exSE4 = remove_utc('Unnamed: 0', 1, exSE4)
exDELUAT = remove_utc('Unnamed: 0', 1, exDELUAT)

In [67]:
# Make a groupby for each hour so it has the same format
exDELU = exDELU.groupby(pd.Grouper(freq='H')).sum()
exDK1 = exDK1.groupby(pd.Grouper(freq='H')).sum()
exSE4 = exSE4.groupby(pd.Grouper(freq='H')).sum()
exDELUAT = exDELUAT.groupby(pd.Grouper(freq='H')).sum()

In [68]:
# Change columns to export specifics
exDELU.rename(columns = {'0':'Export De-Lu'}, inplace = True)
exDK1.rename(columns = {'0':'Export DK1'}, inplace = True)
exSE4.rename(columns = {'0':'Export SE4'}, inplace = True)
exDELUAT.rename(columns = {'0':'Export De-Lu-At'}, inplace = True)

## Investing

In [69]:
# Natural Gas - TTF prices
ttf_prices = pd.read_csv('./data/TTF_prices.csv', decimal = '.')  # in €
ttf_prices['Date'] = pd.to_datetime(ttf_prices['Date'], format = '%m/%d/%Y') # convert date from string to datetime object

# drop unnecessary columns
cols_drop = [col for col in ttf_prices.columns if (col != 'Date' and col != 'Price')]
ttf_prices.drop(cols_drop, axis = 1, inplace = True)

ttf_prices.sort_values(by = ['Date'], ascending = True, inplace = True)
ttf_prices.reset_index(inplace = True, drop = True)

In [70]:
ttf_prices.head()

,Date,Price
0,2018-01-02,19.320
1,2018-01-03,19.325
2,2018-01-04,19.200
3,2018-01-05,18.915
4,2018-01-08,19.050


In [71]:
# CO2 emissions prices
co2_prices = pd.read_csv('./data/CO2_prices.csv', decimal = '.')  # in €
co2_prices['Date'] = pd.to_datetime(co2_prices['Date'], format = '%m/%d/%Y') # convert date from string to datetime object

# drop unnecessary columns
cols_drop = [col for col in co2_prices.columns if (col != 'Date' and col != 'Price')]
co2_prices.drop(cols_drop, axis = 1, inplace = True)

co2_prices.sort_values(by = ['Date'], ascending = True, inplace = True)
co2_prices.reset_index(inplace = True, drop = True)

In [72]:
co2_prices.head()

,Date,Price
0,2018-01-02,8.22
1,2018-01-03,8.24
2,2018-01-04,8.18
3,2018-01-05,8.19
4,2018-01-08,8.07


In [73]:
# coal prices
coal_prices = pd.read_csv('./data/coal_prices.csv', decimal = '.')  # in €
coal_prices['Date'] = pd.to_datetime(coal_prices['Date'], format = '%m/%d/%Y') # convert date from string to datetime object

# drop unnecessary columnsf
cols_drop = [col for col in co2_prices.columns if (col != 'Date' and col != 'Price')]
coal_prices.drop(cols_drop, axis = 1, inplace = True)

coal_prices.sort_values(by = ['Date'], ascending = True, inplace = True)
coal_prices.reset_index(inplace = True, drop = True)

In [74]:
coal_prices.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,2017-12-29,100.80,100.80,100.80,100.80,NaN,-0.40%
1,2018-01-02,102.65,102.65,102.65,102.65,0.01K,1.84%
2,2018-01-03,103.80,103.80,103.80,103.80,NaN,1.12%
3,2018-01-04,104.95,105.00,105.00,105.00,0.05K,1.11%
4,2018-01-05,105.30,105.25,105.50,105.00,0.15K,0.33%


# Pre-processing
## Adjust timeseries

In [75]:
# def remove_utc(col_name, # str: name of the column that contains the object to convert to timestamp
#                tz_offset, # int: timezone offset. E.g., CET = +1
#                df # dataframe: contains all the info
#               ):
#     df['Timestamp'] = pd.to_datetime(df[col_name], format = '%Y %m %d %H:%M:%S',utc = True)
#     df['Timestamp'] = (df['Timestamp'] + dt.timedelta(hours = tz_offset)).dt.tz_localize(None)
#     df.drop([col_name], axis = 1, inplace = True) # drop the column
#     df.set_index('Timestamp', inplace = True) # set column 'Timestamp' as index
#     return df

In [76]:
day_ahead_prices = remove_utc('Unnamed: 0', 1, day_ahead_prices)
day_ahead_prices.tail()

KeyError: 'Unnamed: 0'

In [ ]:
load_and_forecast  = remove_utc('Unnamed: 0', 1, load_and_forecast)
load_and_forecast.tail()

,Forecasted Load,Actual Load
Timestamp,,
2022-12-30 19:00:00,4344.0,4375.0
2022-12-30 20:00:00,4179.0,4187.0
2022-12-30 21:00:00,4053.0,4103.0
2022-12-30 22:00:00,3889.0,3965.0
2022-12-30 23:00:00,3656.0,3750.0


In [ ]:
wind_solar_forecast = remove_utc('Unnamed: 0', 1, wind_solar_forecast)
wind_solar_forecast.tail()

,Solar,Wind Offshore,Wind Onshore
Timestamp,,,
2022-12-30 19:00:00,0.0,1445.0,2973.0
2022-12-30 20:00:00,0.0,1436.0,3033.0
2022-12-30 21:00:00,0.0,1406.0,3205.0
2022-12-30 22:00:00,0.0,1417.0,3272.0
2022-12-30 23:00:00,0.0,1422.0,3275.0


## Join datasets

In [77]:
# create empty dataframe where to "pour" all the data
start = pd.to_datetime('2018-01-01 00:00:00')
end = pd.to_datetime('2023-03-31 23:59:00')
df = pd.DataFrame()
df['Timestamp'] = pd.date_range(start, end, freq = 'H')
df['Date'] = df['Timestamp'].dt.date
df.set_index('Timestamp', inplace = True)# set column 'Timestamp' as index

In [78]:
df.head()

,Date
Timestamp,
2018-01-01 00:00:00,2018-01-01
2018-01-01 01:00:00,2018-01-01
2018-01-01 02:00:00,2018-01-01
2018-01-01 03:00:00,2018-01-01
2018-01-01 04:00:00,2018-01-01


In [79]:
# mapping daily prices: TTF and CO2
df['TTF'] = df['Date'].map(ttf_prices.set_index('Date')['Price'])
df['CO2'] = df['Date'].map(co2_prices.set_index('Date')['Price'])
df['coal'] = df['Date'].map(coal_prices.set_index('Date')['Price'])

In [80]:
df.head()

,Date,TTF,CO2,coal
Timestamp,,,,
2018-01-01 00:00:00,2018-01-01,NaN,NaN,NaN
2018-01-01 01:00:00,2018-01-01,NaN,NaN,NaN
2018-01-01 02:00:00,2018-01-01,NaN,NaN,NaN
2018-01-01 03:00:00,2018-01-01,NaN,NaN,NaN
2018-01-01 04:00:00,2018-01-01,NaN,NaN,NaN


In [81]:
df_merged = pd.merge(df, day_ahead_prices, on = 'Timestamp', how = 'outer')
df_merged = pd.merge(df_merged, load_and_forecast, on = 'Timestamp', how = 'outer')
df_merged = pd.merge(df_merged, wind_solar_forecast, on = 'Timestamp', how = 'outer')

df_merged = pd.merge(df_merged, importshour, on = 'Timestamp', how = 'outer')

df_merged = pd.merge(df_merged, exDELU, on = 'Timestamp', how = 'outer')
df_merged = pd.merge(df_merged, exDK1, on = 'Timestamp', how = 'outer')
df_merged = pd.merge(df_merged, exSE4, on = 'Timestamp', how = 'outer')
df_merged = pd.merge(df_merged, exDELUAT, on = 'Timestamp', how = 'outer')


df_merged.reset_index(inplace = True)

## Saving the resulting dataset

In [82]:
df_merged.to_csv('DK_2.csv', index = False)

In [83]:
df_merged.head()

,Timestamp,Date,TTF,CO2,coal,Day-ahead prices,Forecasted Load,Actual Load,Solar,Wind Offshore,Wind Onshore,DE_AT_LU,DE_LU,DK_1,SE_4,Export De-Lu,Export DK1,Export SE4,Export De-Lu-At
0,2018-01-01 00:00:00,2018-01-01,NaN,NaN,NaN,26.33,3422.0,3421.0,0.0,783.0,1493.0,600.0,0.0,585.0,0.0,NaN,0.0,880.0,0.0
1,2018-01-01 01:00:00,2018-01-01,NaN,NaN,NaN,26.43,3289.0,3308.0,0.0,893.0,1481.0,600.0,0.0,590.0,0.0,NaN,0.0,882.0,0.0
2,2018-01-01 02:00:00,2018-01-01,NaN,NaN,NaN,26.10,3157.0,3118.0,0.0,755.0,1430.0,600.0,0.0,476.0,0.0,NaN,0.0,922.0,0.0
3,2018-01-01 03:00:00,2018-01-01,NaN,NaN,NaN,24.70,3025.0,3018.0,0.0,747.0,1458.0,600.0,0.0,538.0,0.0,NaN,0.0,1087.0,0.0
4,2018-01-01 04:00:00,2018-01-01,NaN,NaN,NaN,24.74,2939.0,2916.0,0.0,886.0,1472.0,600.0,0.0,520.0,0.0,NaN,0.0,1095.0,0.0
